In [1]:
# import torch as th

import matplotlib.pyplot as plt
import time

import sys
import os
from __future__ import division
import os
import urllib, cStringIO
import pymongo as pm ## first establish ssh tunnel to server where database is running
import base64
import numpy as np
from numpy import *
import PIL
from PIL import Image
import base64
import matplotlib
%matplotlib inline
from matplotlib import pylab, mlab, pyplot
from IPython.core.pylabtools import figsize, getfigs
plt = pyplot
import seaborn as sns
sns.set_context('poster')
sns.set_style('white')
from matplotlib.path import Path
import matplotlib.patches as patches
import cv2
import pandas as pd
from svgpathtools import parse_path
import svg_distance_helpers as rsh
# import torch
# from torch.autograd import Variable
# import SimpleITK as sitk
# from torch.autograd import Variable


#### File Hierarchy

In [2]:
# directory & file hierarchy
analysis_dir = os.getcwd()
data_dir = os.path.join(analysis_dir,"tracing_overlap")
ref_dir = os.path.join(analysis_dir, 'tracing_ref_400')
tracing = ['square','shape', 'this circle']

#### Set Up

In [3]:
# generate reference shapes
really_run_this = 1
if really_run_this:
    ref_path = []
    for t in tracing:
        print t
        fname = os.path.join(ref_dir, '{}.png'.format(t))
        img_ref = cv2.imread(fname)
        img_ref = rsh.color_to_num(img_ref, threshold=0, white_background=False)
        ofname = os.path.join(ref_dir, '{}_ref.png'.format(t))
        cv2.imwrite(ofname, 255-img_ref*255)
        ref_path.append(ofname)
    

square
shape
this circle


In [ ]:
# Overlap the drawing on reference images
iteration_name = ['CDM_photodraw_e1', 'cdm_run_v3', 'cdm_run_v4']# name labels used in kiddraw database
dir_name = ['photodraw_e1', 'museumstation_v3', 'museumstation_v4'] # name labels will be used for new data storage 
tracing = ['square', 'shape', 'this circle']

# create the folder to store overlapped images
overlap_dir = os.path.join(analysis_dir, 'tracing_overlap')
if not os.path.exists(overlap_dir):
    os.makedirs(overlap_dir)

for di, dname in enumerate(dir_name):
    # create an individual folder for each experiment+version pair
    iter_dir = os.path.join(overlap_dir, dname)
    if not os.path.exists(iter_dir):
        os.makedirs(iter_dir)
        
    current_data_dir = os.path.join(analysis_dir,"tracing_{}".format(iteration_name[di]))

    # get tracings from the data folder
    for ti, t in enumerate(tracing):
            
        # folders that store the original tracing files
        if dname == 'photodraw_e1' and (t == 'square' or t == 'shape'):
            t = 'this ' + t
        t_dir = os.path.join(current_data_dir, t)
        
        # create folders to store overlapped images
        overlap_t = os.path.join(iter_dir, t.replace('this ', ''))
        if not os.path.exists(overlap_t):
                os.makedirs(overlap_t)
        
        
        if os.path.exists(t_dir):
            for fname in os.listdir(t_dir):
                if fname.startswith('age'):
                    img_draw = os.path.join(t_dir, fname)
                    img_ref = ref_path[ti]

                    draw = cv2.imread(img_draw).astype(int16)
                    ref = cv2.imread(img_ref).astype(int16)
                    img = ref
                    img[np.sum(ref, axis=2)<255*3] = [150, 150, 150]
                    img[np.sum(draw, axis=2)<255*3] = [40, 40, 170]

                    split_fname = fname.split('_')
                    age = split_fname[0]
                    timestamp = split_fname[3][2:]
                    new_fname = "{}_{}_{}_{}.png".format(age, dname, timestamp, t.replace('this ', ''))
                    cv2.imwrite(os.path.join(iter_dir,t.replace('this ', ''), new_fname), img)

#### upload stims to s3

Execute python script at command line by running: `python upload_stims_to_s3.py --bucket_name kiddraw-tracing --path_to_imgs ./tracing_eval_stims`

#### build stimulus dictionary & upload metadata to mongo

In [ ]:
# read in data from csv files
cdm_v3 = pd.read_csv('MuseumStation_Tracing_Descriptives_cdm_run_v3.csv')
cdm_v4 = pd.read_csv('MuseumStation_Tracing_Descriptives_cdm_run_v4.csv')
photodraw = pd.read_csv('Photodraw_Tracing_Descriptives_CDM_photodraw_e1.csv')
limit = 200
overall = photodraw
#overall = pd.concat([overall, cdm_v3, cdm_v4])

for t in tracing:
    for a in range(2,11):
        cdm_v3_tage = cdm_v3.loc[(cdm_v3['age'] == 'age{}'.format(a)) & (cdm_v3['category']== t)]
        v3_size = cdm_v3_tage.shape[0]                 
        if v3_size > limit:
            overall = pd.concat([overall, cdm_v3_tage[:limit]])
            
        else:
            overall = pd.concat([overall, cdm_v3_tage]) 
            
            cdm_v4_tage = cdm_v4.loc[(cdm_v4['age'] == 'age{}'.format(a)) & (cdm_v4['category']== t)]
            
            overall = pd.concat([overall, cdm_v4_tage[:limit-v3_size]])

print overall.shape
for t in tracing:
    for a in range(2,11):
        print t, a, overall.loc[(overall['age'] == 'age{}'.format(a)) & ( (overall['category'] == t) | (overall['category'] == 'this {}'.format(t)) )].shape

overall = overall.drop(['filename', "Unnamed: 0"], axis=1)
overall.reset_index(inplace = True)

In [ ]:
overall['exp_name'] = 0
overall['iteration_name'] = 0
overall['timestamp'] = 0
overall['img_url'] = 0
overall['number_rating_levels'] = 5
overall['lower_bound'] = 'poor'
overall['upper_bound'] = 'excellent' 

for index, row in overall.iterrows():
    sid = row['session_id']    
    
    if sid.startswith('CDM'):
        overall.loc[index,'exp_name'] = 'photodraw'
        overall.loc[index,'iteration_name'] = 'e1'
    else:
        overall.loc[index,'exp_name'] = 'museumstation'
        overall.loc[index,'iteration_name'] = sid.split('_')[2][:2]
    
    timestamp = row['session_id'].split('_')[-1][2:]
    overall.loc[index, 'timestamp'] = timestamp
    category = row['category'].replace('this ','')
    overall.loc[index,'category'] = category
    
    fname = '{}_{}_{}_{}_{}.png'.format(row['age'], overall.loc[index,'exp_name'],overall.loc[index,'iteration_name'], timestamp, category)
    
    folder_name = '{}_{}'.format(overall.loc[index,'exp_name'], overall.loc[index,'iteration_name'])
    render_fname = os.path.join(data_dir, folder_name, category, fname)
    
    if not os.path.exists(render_fname):
        print index, render_fname
        overall = overall.drop(index)
        continue
    
    overall.loc[index,'img_url'] = 'https://s3.amazonaws.com/{}/{}'.format('kiddraw-tracing',fname)
    overall.loc[index,'games'] = '[]'
    overall.loc[index,'age'] = overall.loc[index,'age'].replace('age','')

overall['age'] = pd.to_numeric(overall['age'])


In [ ]:
try:
    overall = overall.drop('index', axis=1)
except:
    pass

bin_limit = 80
for t in ['square', 'shape', 'circle']:
    for a in range(2,11):
        drop_index = overall.index[(overall['age'] == a) & (overall['category'] == t)].tolist()[bin_limit:]
        overall = overall.drop(drop_index)
        print t, a, overall.loc[(overall['age'] == a) & ( (overall['category'] == t) | (overall['category'] == 'this {}'.format(t)) )].shape



In [ ]:
## apply weird literal eval hack to convert string into regular list
import ast
overall['games'] = '[]'
overall['games'] = overall['games'].apply(lambda x: ast.literal_eval(x))
print overall.shape

In [ ]:
# insert shuffler_ind
data_range = np.arange(overall.shape[0])
np.random.shuffle(data_range)
overall['shuffler_ind'] = data_range

In [ ]:
try:
    overall = overall.drop('index', axis=1)
except:
    pass

## save out data frame as csv for our own records
overall.to_csv('kiddraw_tracing_eval.csv',index=False)

In [ ]:
## convert 'overall' pandas dataframe to json object and save out
import json
stimdict = overall.to_dict(orient='records') 
with open('{}.js'.format('kiddraw_tracing_eval'), 'w') as fout:
    json.dump(stimdict, fout)

### load JSON back in to inspect before uploading to mongo

In [ ]:
### define datasetname
dataset_name = 'kiddraw_tracing_eval'

### next todo is to upload this JSON to initialize the new stimulus collection
print('next todo is to upload this JSON to initialize the new stimulus collection...')
import json
J = json.loads(open('{}.js'.format(dataset_name),mode='ru').read())

In [ ]:
##assert len(J)==len(all_files)
print 'dataset_name: {}'.format(dataset_name)
print len(J)

In [ ]:
## look at one example record
J[0]

In [ ]:
# set vars 
auth = pd.read_csv('auth.txt', header = None) # this auth.txt file contains the password for the sketchloop user
pswd = auth.values[0][0]
user = 'sketchloop'
host = 'rxdhawkins.me' ## cocolab ip address

# have to fix this to be able to analyze from local
conn = pm.MongoClient('mongodb://sketchloop:' + pswd + '@127.0.0.1')
db = conn['stimuli']
coll = db[dataset_name]
dev_name = 'kiddraw_tracing_eval_dev'
dev_coll = db[dev_name]

In [ ]:
reallyRun = 1
if reallyRun:
    coll.drop()
print 'Currently there are {} tracings in the production collection'.format(coll.count())

In [ ]:
from IPython.display import clear_output
## actually add data now to the database
reallyRun = 1
if reallyRun:
    for (i,j) in enumerate(J):
        if i%1==0:
            print ('%d of %d' % (i,len(J)))
            clear_output(wait=True)
        coll.insert(j)

In [ ]:
reallyRun = 1
if reallyRun:
    dev_coll.drop()
print 'Currently there are {} tracings in the development collection'.format(dev_coll.count())

In [ ]:
# set the dev collection
reallyRun = 1
if reallyRun:
    for (i,j) in enumerate(J):
        if i%1==0:
            print ('%d of %d' % (i,len(J)))
            clear_output(wait=True)
        dev_coll.insert(j)

In [ ]:
## check how many records have been retrieved
print 'Currently there are {} tracings in the production collection'.format(coll.count())
print 'Currently there are {} tracings in the development collection'.format(dev_coll.count())

a = coll.find({'num_strokes':{'$gte':0}}).sort('timestamp')
numGames = []
for rec in a:
    numGames.append(len(rec['games']))
b = np.array(numGames)
print np.mean(b>0)

In [ ]:
print "Currentely, there are {} number of tracing stimuli in the database".format(coll.count())
coll.find_one()